Model Summary

In [ ]:
import torch
from model import Net
from torchinfo import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)
batch_size = 2
summary(
    model,
    input_size=(batch_size, 3, 32, 32),
    depth=4,
    col_width=18,
    col_names=[
        "kernel_size",
        "input_size",
        "output_size",
        "num_params",
        "trainable",
    ],
    row_settings=["var_names"],
)

Data loader

In [ ]:
from dataloader import Cifar10DataLoader

cifar10 = Cifar10DataLoader(is_cuda_available=use_cuda)
train_loader = cifar10.get_loader(True)
test_loader = cifar10.get_loader(False)

Training

In [ ]:
import torch.optim as optim
import torch.nn as nn
from train import Trainer
from test import Tester

trainer = Trainer()
tester = Tester()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.NLLLoss()

EPOCHS = 200

for epoch in range(EPOCHS):
    trainer.train(model, train_loader, optimizer, criterion, device, epoch)
    tester.test(model, test_loader, criterion, device)

Stats

In [ ]:
print("Minimum training loss = {:.9f}%".format(min(trainer.train_losses)))
print("Minimum testing loss = {:.9f}%".format(min(tester.test_losses)))
print("Best training accuracy = {:.2f}%".format(max(trainer.epoch_train_accuracies)))
print("Best testing accuracy = {:.2f}%".format(max(tester.test_accuracies)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def denormalize(img):
    channel_means = (0.4914, 0.4822, 0.4465)
    channel_stdevs = (0.2470, 0.2435, 0.2616)
    img = img.astype(dtype=np.float32)

    for i in range(img.shape[0]):
        img[i] = (img[i] * channel_stdevs[i]) + channel_means[i]

    return np.transpose(img, (1, 2, 0))

def show_training_images(train_loader, count):
    classes = (
            "plane",
            "car",
            "bird",
            "cat",
            "deer",
            "dog",
            "frog",
            "horse",
            "ship",
            "truck",
    )

    images, labels = next(iter(train_loader))
    images = images[0:count]
    labels = labels[0:count]

    fig = plt.figure(figsize=(20, 10))
    for i in range(count):
        sub = fig.add_subplot(count/5, 5, i+1)
        npimg = denormalize(images[i].cpu().numpy().squeeze())
        plt.imshow(npimg, cmap="gray")
        sub.set_title("Correct class: {}".format(classes[labels[i]]))
    plt.tight_layout()
    plt.show()

Training Images

In [ ]:
show_training_images(train_loader, 20)

Code to show misclassified images

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def show_misclassified_images(model, test_loader, device):
    model.eval()
    classes = (
            "plane",
            "car",
            "bird",
            "cat",
            "deer",
            "dog",
            "frog",
            "horse",
            "ship",
            "truck",
    )
    misclassified_images = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, pred = torch.max(output, 1)
            for i in range(len(pred)):
                if pred[i] != target[i]:
                    misclassified_images.append({'image': data[i], 'predicted_class': classes[pred[i]], 'correct_class': classes[target[i]]})

    # Plot the misclassified images
    fig = plt.figure(figsize=(20, 10))
    for i in range(20):
        sub = fig.add_subplot(4, 5, i+1)
        misclassified_image = misclassified_images[i]
        npimg = denormalize(misclassified_image['image'].cpu().numpy().squeeze())
        plt.imshow(npimg, cmap="gray")
        sub.set_title("Correct class: {}\nPredicted class: {}".format(misclassified_image['correct_class'], misclassified_image['predicted_class']))
    plt.tight_layout()
    plt.show()

misclassified images

In [ ]:
show_misclassified_images(model, test_loader, device)

Plots

In [ ]:
fig, ax = plt.subplots(2, 2)

train_epoch_loss_linspace = np.linspace(0, EPOCHS, len(trainer.train_losses))
test_epoch_loss_linspace = np.linspace(0, EPOCHS, len(tester.test_losses))
train_epoch_acc_linspace = np.linspace(0, EPOCHS, len(trainer.train_accuracies))
test_epoch_acc_linspace = np.linspace(0, EPOCHS, len(tester.test_accuracies))

ax[0][0].set_xlabel("Epoch")
ax[0][0].set_ylabel("Train Loss")
ax[0][0].plot(train_epoch_loss_linspace, trainer.train_losses)
ax[0][0].tick_params(axis="y", labelleft=True, labelright=True)

ax[0][1].set_xlabel("Epoch")
ax[0][1].set_ylabel("Test Loss")
ax[0][1].plot(test_epoch_loss_linspace, tester.test_losses)
ax[0][1].tick_params(axis="y", labelleft=True, labelright=True)

ax[1][0].set_xlabel("Epoch")
ax[1][0].set_ylabel("Train Accuracy")
ax[1][0].plot(train_epoch_acc_linspace, trainer.train_accuracies)
ax[1][0].tick_params(axis="y", labelleft=True, labelright=True)
ax[1][0].yaxis.set_ticks(np.arange(0, 101, 5))

ax[1][1].set_xlabel("Epoch")
ax[1][1].set_ylabel("Test Accuracy")
ax[1][1].plot(test_epoch_acc_linspace, tester.test_accuracies)
ax[1][1].tick_params(axis="y", labelleft=True, labelright=True)
ax[1][1].yaxis.set_ticks(np.arange(0, 101, 5))

fig.set_size_inches(28, 18)
plt.tight_layout()
plt.show()